# Project A: Knowledge Distillation for Building Lightweight Deep Learning Models in Visual Classification Tasks

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Utilities import *

# Data loading

In [2]:
mnist_train, mnist_test = loadMnist()

# Model creation

In [3]:
teacherModel = getTeacherModel()
studenModel = getStudentModel()
train_and_evaluate(teacherModel,mnist_train,mnist_test)
teacherTestAcc = testModel(teacherModel,mnist_test)
train_and_evaluate_using_KD(studenModel,teacherModel,mnist_train,mnist_test,0.5,4)
studentTestAcc = testModel(studenModel,mnist_test)

Epoch 1: Class_accuracy: 98.33%
Epoch 2: Class_accuracy: 98.67%
Epoch 3: Class_accuracy: 98.87%
Epoch 4: Class_accuracy: 99.00%
Epoch 5: Class_accuracy: 98.98%
Epoch 6: Class_accuracy: 98.97%
Epoch 7: Class_accuracy: 99.17%
Epoch 8: Class_accuracy: 99.13%
Epoch 9: Class_accuracy: 99.16%
Epoch 10: Class_accuracy: 99.19%
Epoch 11: Class_accuracy: 99.12%
Epoch 12: Class_accuracy: 99.26%
model Testing Accuracy: 99.26%
Epoch 1: Class_accuracy: 96.65%
Epoch 2: Class_accuracy: 97.35%
Epoch 3: Class_accuracy: 97.74%
Epoch 4: Class_accuracy: 98.01%
Epoch 5: Class_accuracy: 97.69%
Epoch 6: Class_accuracy: 97.82%
Epoch 7: Class_accuracy: 98.18%
Epoch 8: Class_accuracy: 98.20%
Epoch 9: Class_accuracy: 98.28%
Epoch 10: Class_accuracy: 98.12%
Epoch 11: Class_accuracy: 97.78%
Epoch 12: Class_accuracy: 97.97%
model Testing Accuracy: 97.97%


# Test accuracy vs. tempreture curve

In [4]:
# testACC = []
# tempratureValues = [1, 2, 4, 16, 32, 64]
# for temp in tempratureValues:
#     st = getStudentModel()
#     train_and_evaluate_using_KD(st,teacherModel,mnist_train,mnist_test,0.5, temp)
#     testACC += [testModel(st,mnist_test)]

# Train student from scratch

In [5]:
fc_model_no_distillation = getStudentModel()
train_and_evaluate(fc_model_no_distillation,mnist_train,mnist_test)
teacherTestAcc = testModel(fc_model_no_distillation,mnist_test)


Epoch 1: Class_accuracy: 97.17%
Epoch 2: Class_accuracy: 97.58%
Epoch 3: Class_accuracy: 97.73%
Epoch 4: Class_accuracy: 97.66%
Epoch 5: Class_accuracy: 98.00%
Epoch 6: Class_accuracy: 98.12%
Epoch 7: Class_accuracy: 97.90%
Epoch 8: Class_accuracy: 98.11%
Epoch 9: Class_accuracy: 97.53%
Epoch 10: Class_accuracy: 98.06%
Epoch 11: Class_accuracy: 98.04%
Epoch 12: Class_accuracy: 98.38%
model Testing Accuracy: 98.38%


# Comparing the teacher and student model (number of of parameters and FLOPs) 

In [6]:
# your code start from here for step 8


# Implementing the state-of-the-art KD algorithm

In [7]:
teachingAssistant = getTeachingAssistantModel()
train_and_evaluate_using_KD(teachingAssistant,teacherModel,mnist_train,mnist_test,0.5,4)
teachingAssistantAcc = testModel(teachingAssistant,mnist_test)
student = getStudentModel()
train_and_evaluate_using_KD(student,teachingAssistant,mnist_train,mnist_test,0.5,4)
studenttAcc = testModel(student,mnist_test)

Epoch 1: Class_accuracy: 96.25%
Epoch 2: Class_accuracy: 97.54%
Epoch 3: Class_accuracy: 97.60%
Epoch 4: Class_accuracy: 98.00%
Epoch 5: Class_accuracy: 98.07%
Epoch 6: Class_accuracy: 97.99%
Epoch 7: Class_accuracy: 98.13%
Epoch 8: Class_accuracy: 98.26%
Epoch 9: Class_accuracy: 98.16%
Epoch 10: Class_accuracy: 98.11%
Epoch 11: Class_accuracy: 97.84%
Epoch 12: Class_accuracy: 98.29%
model Testing Accuracy: 98.29%
Epoch 1: Class_accuracy: 96.71%
Epoch 2: Class_accuracy: 97.05%
Epoch 3: Class_accuracy: 97.61%
Epoch 4: Class_accuracy: 98.09%
Epoch 5: Class_accuracy: 97.79%
Epoch 6: Class_accuracy: 98.06%
Epoch 7: Class_accuracy: 97.91%
Epoch 8: Class_accuracy: 97.85%
Epoch 9: Class_accuracy: 98.31%
Epoch 10: Class_accuracy: 98.20%
Epoch 11: Class_accuracy: 98.06%
Epoch 12: Class_accuracy: 97.88%
model Testing Accuracy: 97.88%


# (Optional) XAI method to explain models

In [8]:
# your code start from here for step 13
